In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
import wandb
import os
import random
from time import time

from lib.data import EpisodeDataset
from lib.neural_networks import FullyConnected3, ConvNet

In [2]:
WIDTH = 10
OBS_SIZE = 46
HIDDEN_SIZE = 512

In [3]:
TRAIN_PATH_1 = '/home/bdemoss/Desktop/RL IDM/Episode Data/Random Data/30 TPS/3v3 Train'
TRAIN_PATH_2 = '/home/bdemoss/Desktop/RL IDM/Episode Data/Nexto Data/30 TPS/3v3 Train'

train_dataset = EpisodeDataset([TRAIN_PATH_1, TRAIN_PATH_2], width=WIDTH, include_change=True, corrupt=True)

print(len(train_dataset))

Loading Episode Data
Finished Loading Episode Data
311700


In [4]:
Train_Data_Loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=16)

In [5]:
#model = ConvNet(obs_size=OBS_SIZE, obs_width=WIDTH, conv_number=10, hidden_size=HIDDEN_SIZE)
model = FullyConnected3(obs_size=OBS_SIZE, obs_width=WIDTH, hidden_size=HIDDEN_SIZE)

gpumodel = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpumodel.parameters(), lr=0.0001)

In [53]:
gpumodel.load_state_dict(torch.load('trained_networks/fc3_pitch_512x4_w_change_corrupt.pt'))

<All keys matched successfully>

In [6]:
wandb.init(project="Inverse Dynamics Model", entity="harrymead")

wandb.config = {
  "learning_rate": 0.0001,
  "epochs": 10,
  "batch_size": 10
}

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: harrymead. Use `wandb login --relogin` to force relogin


In [7]:
gpumodel.train()

for epoch in range(10):
    batch = 0
    total_loss = 0
    obs_count = 0
    total_correct = torch.zeros(9).cuda()

    for _obs, _act, _add_data in Train_Data_Loader:
        # TRAINING 
        obs = _obs[0].cuda()
        act = torch.cat((_act[0], _add_data[0][:,0:1]), dim=1).long().cuda()
        add_data = _add_data[0]

        if obs.shape[0] == 0:
            continue

        optimizer.zero_grad()

        y_pred = model(obs)
        pred_act = torch.zeros(obs.shape[0], 9).cuda()

        loss = 0
        for i in range(9):
            loss += criterion(y_pred[i], act[:, i])

            pred_act[:, i] = torch.argmax(y_pred[i], dim=1)

        loss.backward()
        optimizer.step()


        # LOGGING
        obs_count += len(obs)
        total_loss += loss.item() * len(obs)

        correct = act == pred_act
        total_correct += torch.sum(correct, dim=0)


        batch += 1
        if batch % 1000 == 0:
            idv_accuracy = total_correct/obs_count

            print(f'Epoch {epoch + 1} - Loss: {total_loss / obs_count}, Correct: {idv_accuracy}')
            wandb.log({"loss": total_loss/obs_count, "accuracy/throttle": idv_accuracy[0], "accuracy/steer": idv_accuracy[1], "accuracy/yaw": idv_accuracy[2], 
                        "accuracy/pitch": idv_accuracy[3], "accuracy/roll": idv_accuracy[4], "accuracy/jump": idv_accuracy[5], "accuracy/boost": idv_accuracy[6],
                        "accuracy/handbrake": idv_accuracy[7], "accuracy/on_ground": idv_accuracy[8]})

            total_loss = 0
            obs_count = 0
            total_correct = torch.zeros(9).cuda()

Epoch 1 - Loss: 3.98425845706487, Correct: tensor([0.7304, 0.6719, 0.8246, 0.6727, 0.8435, 0.9260, 0.8756, 0.9645, 0.9109],
       device='cuda:0')
Epoch 1 - Loss: 2.95541998136324, Correct: tensor([0.8474, 0.7410, 0.8571, 0.7420, 0.8758, 0.9343, 0.9447, 0.9698, 0.9724],
       device='cuda:0')
Epoch 1 - Loss: 2.6510093987292582, Correct: tensor([0.8825, 0.7549, 0.8674, 0.7553, 0.8860, 0.9422, 0.9550, 0.9716, 0.9790],
       device='cuda:0')
Epoch 1 - Loss: 2.4865594602238756, Correct: tensor([0.9020, 0.7624, 0.8789, 0.7628, 0.8864, 0.9365, 0.9595, 0.9709, 0.9844],
       device='cuda:0')
Epoch 1 - Loss: 2.3790626871917175, Correct: tensor([0.9127, 0.7690, 0.8842, 0.7699, 0.8883, 0.9418, 0.9623, 0.9727, 0.9852],
       device='cuda:0')
Epoch 1 - Loss: 2.2553151434715555, Correct: tensor([0.9223, 0.7701, 0.8940, 0.7700, 0.8956, 0.9446, 0.9662, 0.9704, 0.9871],
       device='cuda:0')
Epoch 1 - Loss: 2.1838434728977747, Correct: tensor([0.9239, 0.7787, 0.8974, 0.7803, 0.8978, 0.9446, 0.9

KeyboardInterrupt: 

In [38]:
torch.save(gpumodel.state_dict(), "trained_networks/fc3_all_512x4_w_change_corrupt_w_og.pt")

In [64]:
#TEST_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Nexto Data/30 TPS/3v3 Test'
#TEST_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Human Example Data/30 TPS'
TEST_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Human Replays'

test_dataset = EpisodeDataset([TEST_PATH], width=WIDTH, include_change=True, corrupt=False)

Loading Episode Data
Finished Loading Episode Data


In [65]:
Test_Data_Loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [66]:
conf_mtx = torch.zeros((8,3,3)).cuda()
CONF_MAT = True

gpumodel.eval()

total_loss = 0
obs_count = 0
total_correct = torch.zeros(9).cuda()

correct_on_ground = torch.zeros(9).cuda()
total_on_ground = 0

for _obs, _act, _add_data in Test_Data_Loader:
    # EVALUATION
    obs = _obs[0, 300:1600].cuda()
    act = torch.cat((_act[0, 300:1600], _add_data[0, 300:1600][:,0:1]), dim=1).long().cuda()
    add_data = _add_data[0].long()

    if obs.shape[0] == 0:
        continue

    with torch.no_grad():
        y_pred = model(obs)

    y_pred = model(obs)
    pred_act = torch.zeros(obs.shape[0], 9).cuda()

    loss = 0
    for i in range(9):
        loss += criterion(y_pred[i], act[:, i])

        pred_act[:, i] = torch.argmax(y_pred[i], dim=1)


    # LOGGING
    obs_count += len(obs)
    total_loss += loss.item() * len(obs)

    correct = act == pred_act
    total_correct += torch.sum(correct, dim=0)

    on_ground = pred_act[:,8]

    #print(correct)
    #print(on_ground.bool())

    total_on_ground += torch.sum(on_ground == 1).item()


    correct_on_ground += torch.sum(correct[on_ground == 1], dim=0)

    if CONF_MAT:
        for i in range(obs.shape[0]):
            for j in range(8):
                if on_ground[i] == 1:
                    conf_mtx[j, act[i, j], pred_act[i, j].long()] += 1

    

print(f'Loss: {total_loss / obs_count}, Correct: {total_correct / obs_count}, Correct On Ground: {correct_on_ground / total_on_ground}, On Ground: {total_on_ground / obs_count}')

np.set_printoptions(precision=4, suppress=True)
print((conf_mtx).cpu().numpy())

Loss: 0.6540960669517517, Correct: tensor([0.9746, 0.9692, 0.9531, 0.9677, 0.9892, 0.9523, 0.9946, 0.9808, 0.9938],
       device='cuda:0'), Correct On Ground: tensor([0.9656, 0.9828, 0.9977, 0.9793, 1.0000, 0.9989, 0.9931, 0.9713, 0.9954],
       device='cuda:0'), On Ground: 0.67
[[[ 34.   0.   0.]
  [  0.  83.   3.]
  [  1.  26. 724.]]

 [[229.   0.   1.]
  [  2. 303.   3.]
  [  0.   9. 324.]]

 [[  0.   1.   0.]
  [  0. 869.   0.]
  [  0.   1.   0.]]

 [[225.   1.   4.]
  [  2. 298.   8.]
  [  0.   3. 330.]]

 [[  0.   0.   0.]
  [  0. 871.   0.]
  [  0.   0.   0.]]

 [[857.   1.   0.]
  [  0.  13.   0.]
  [  0.   0.   0.]]

 [[687.   5.   0.]
  [  1. 178.   0.]
  [  0.   0.   0.]]

 [[823.   5.   0.]
  [ 20.  23.   0.]
  [  0.   0.   0.]]]
